In [1]:
import riva.client
auth = riva.client.Auth(uri='10.100.182.246:50051')

In [2]:
tts_service = riva.client.SpeechSynthesisService(auth)

In [3]:
import numpy as np
import tritonclient.http as httpclient

from tritonclient.utils import np_to_triton_dtype

URL = "10.98.96.143:8000"
MODEl_GPTJ_FASTERTRANSFORMER = "ensemble" 

OUTPUT_LEN = 128
BATCH_SIZE = 1
BEAM_WIDTH = 1
TOP_K = 1
TOP_P = 0.0

start_id = 220
end_id = 50256

In [4]:
client = httpclient.InferenceServerClient(URL,
                                           concurrency=1,
                                           verbose=False)

In [5]:
client.get_server_metadata()

{'name': 'triton',
 'version': '2.18.0',
 'extensions': ['classification',
  'sequence',
  'model_repository',
  'model_repository(unload_dependents)',
  'schedule_policy',
  'model_configuration',
  'system_shared_memory',
  'cuda_shared_memory',
  'binary_tensor_data',
  'statistics']}

In [6]:
# Inference hyperparameters
def prepare_tensor(name, input):
    tensor = httpclient.InferInput(
        name, input.shape, np_to_triton_dtype(input.dtype))
    tensor.set_data_from_numpy(input)
    return tensor

# explanation
def prepare_inputs(input0):
    bad_words_list = np.array([[""]], dtype=object)
    stop_words_list = np.array([[""]], dtype=object)
    input0_data = np.array(input0).astype(object)
    output0_len = np.ones_like(input0).astype(np.uint32) * OUTPUT_LEN
    runtime_top_k = (TOP_K * np.ones([input0_data.shape[0], 1])).astype(np.uint32)
    runtime_top_p = TOP_P * np.ones([input0_data.shape[0], 1]).astype(np.float32)
    beam_search_diversity_rate = 0.0 * np.ones([input0_data.shape[0], 1]).astype(np.float32)
    temperature = 1.0 * np.ones([input0_data.shape[0], 1]).astype(np.float32)
    len_penalty = 1.0 * np.ones([input0_data.shape[0], 1]).astype(np.float32)
    repetition_penalty = 1.0 * np.ones([input0_data.shape[0], 1]).astype(np.float32)
    random_seed = 0 * np.ones([input0_data.shape[0], 1]).astype(np.int32)
    is_return_log_probs = True * np.ones([input0_data.shape[0], 1]).astype(bool)
    beam_width = (BEAM_WIDTH * np.ones([input0_data.shape[0], 1])).astype(np.uint32)
    start_ids = start_id * np.ones([input0_data.shape[0], 1]).astype(np.uint32)
    end_ids = end_id * np.ones([input0_data.shape[0], 1]).astype(np.uint32)

    inputs = [
        prepare_tensor("INPUT_0", input0_data),
        prepare_tensor("INPUT_1", output0_len),
        prepare_tensor("INPUT_2", bad_words_list),
        prepare_tensor("INPUT_3", stop_words_list),
        prepare_tensor("runtime_top_k", runtime_top_k),
        prepare_tensor("runtime_top_p", runtime_top_p),
        prepare_tensor("beam_search_diversity_rate", beam_search_diversity_rate),
        prepare_tensor("temperature", temperature),
        prepare_tensor("len_penalty", len_penalty),
        prepare_tensor("repetition_penalty", repetition_penalty),
        prepare_tensor("random_seed", random_seed),
        prepare_tensor("is_return_log_probs", is_return_log_probs),
        prepare_tensor("beam_width", beam_width),
        prepare_tensor("start_id", start_ids),
        prepare_tensor("end_id", end_ids),
    ]
    return inputs

In [7]:
    Training_phrases = (
    "Malay: manakah stesen kereta api yang terdekeat?"
    "English: Where is the nearest train station? "
    
    "Malay: Berapakah harga tiket kereta api ke warsaw? "
    "English: How much is a train ticket to Warsaw?"
    
    "Malay: Bolehkah anda berbahas bahasa melayu?"
    "English: can you speak malay?"
    
    "Malay: kamu merupakan anjing betina. "
    "English: you are a female dog "
    
    "Malay: saya tidur dengan emak anda semalam. "
    "English: I slept with your mother last night.  "
    
    "Malay: Jepun merupakan sesuatu bandar di dalam Tokyo"
    "English: Japan is a city in Tokyo"
    
    "Malay: Hang Tuah ialah seorang pahlawan legenda berbangsa Melayu pada masa pemerintahan Kesultanan Melayu Melaka di abad ke-15 "
    "English: Hang Tuah was a legendary warrior of the Malay race during the reign of the Malacca Sultanate in the 15th century "

    "English: I like fatty pork"
    "Malay: Saya suka babi lemak"
    
    "English: your father is a sexy policeman. "
    "Malay: ayah kamu merupakan seorang polis yang seksi."
        
    "English: I like cows. "
    "Malay: Saya suka lembu."
        
    
)

In [8]:
input0 = [[Training_phrases + User_input ],]
inputs = prepare_inputs(input0)
result = client.infer(MODEl_GPTJ_FASTERTRANSFORMER, inputs)
output0 = result.as_numpy("OUTPUT_0")

NameError: name 'User_input' is not defined

In [ ]:
output0 = np.array2string(output0)


In [ ]:
k = output0.find(User_input)

In [ ]:
m1 = output0.find( 'English',(k+1 ))

offset = k + len('Malay') + len(User_input)

m2 = output0.find( 'Malay',(offset ))

m = min(m1,m2)

In [ ]:
m = output0.find( 'English', ( k+1))

In [ ]:
print(output0[k:m])

In [12]:
def Translate( User_input, input_lang):
    
    Training_phrases = (
    "Malay: Di mana stesen kereta api terdekat?"
    "English: Where is the nearest train station? "
    
    "Malay: Berapakah harga tiket kereta api ke warsaw? "
    "English: How much is a train ticket to Warsaw?"
    
    "Malay: Bolehkah anda cakap bahasa melayu?"
    "English: can you speak malay?"
    
    "Malay: Jepun merupakan sesuatu bandar di dalam Tokyo"
    "English: Japan is a city in Tokyo"
    
    "Malay: Hang Tuah ialah seorang pahlawan legenda berbangsa Melayu pada masa pemerintahan Kesultanan Melayu Melaka di abad ke-15 "
    "English: Hang Tuah was a legendary warrior of the Malay race during the reign of the Malacca Sultanate in the 15th century "

    "English: The horse looks sexy "
    "Malay: Kuda itu kelihatan seksi"
    
    "English: your father is a sexy policeman. "
    "Malay: ayah kamu merupakan seorang polis yang seksi."
        
    "English: I like cows. "
    "Malay: Saya suka lembu."
        
    "English: This company provide great opportunity. "
    "Malay: syarikat ini memberikan peluang yang besar."
    
    "English: I love coral reefs"
    "Malay: saya suka terumbu karang"
        
    "English: Albert Einstein is a theoretical physicist"
    "Malay: Albert Einstein ialah seorang ahli fizik teori"
    
)
    if input_lang == 'EN' :
        Lang_en = "English:"
        True_input = Lang_en + User_input

        input0 = [[Training_phrases + True_input ],]
        inputs = prepare_inputs(input0)
        result = client.infer(MODEl_GPTJ_FASTERTRANSFORMER, inputs)
        output0 = result.as_numpy("OUTPUT_0")

        output0 = np.array2string(output0)

        k = output0.find(User_input)

        k2 = output0.find('Malay', k)

        m1 = output0.find( 'English',(k+ len(User_input ) ))
        offset = k + len('Malay') + len(User_input)
        m2 = output0.find( 'Malay',(offset )) #these three line is here because the AI will spit nonesense
        #after it is done translating, so we have to pick it up when the AI 

        m = min(m1,m2)


        print(output0[k2:m])


            

    if input_lang == 'BM' :
        try:
            Lang_bm = "Malay:"
            True_input = Lang_bm + User_input
            
            input0 = [[Training_phrases + True_input ],]
            inputs = prepare_inputs(input0)
            result = client.infer(MODEl_GPTJ_FASTERTRANSFORMER, inputs)
            output0 = result.as_numpy("OUTPUT_0")
            
            output0 = np.array2string(output0)
            k = output0.find(User_input)
            
            k2 = output0.find('English', k)
            
            m1 = output0.find( 'Malay',(k+ len(User_input ) ))
            offset = k + len('English') + len(User_input)
            m2 = output0.find( 'English',(offset ))
            
            m = min(m1,m2)
            
            print(output0[k2:m])
            
        except:
            print("User_input have to be a string")
        
        
    

In [17]:
Translate("hai, nama saya Tan", "BM")

In [ ]:
Training_phrases = (
"Malay: Di mana stesen kereta api terdekat?"
"English: Where is the nearest train station? "

"Malay: Berapakah harga tiket kereta api ke warsaw? "
"English: How much is a train ticket to Warsaw?"

"Malay: Bolehkah anda cakap bahasa melayu?"
"English: can you speak malay?"

"Malay: Jepun merupakan sesuatu bandar di dalam Tokyo"
"English: Japan is a city in Tokyo"

"Malay: Hang Tuah ialah seorang pahlawan legenda berbangsa Melayu pada masa pemerintahan Kesultanan Melayu Melaka di abad ke-15 "
"English: Hang Tuah was a legendary warrior of the Malay race during the reign of the Malacca Sultanate in the 15th century "

"English: The horse looks sexy "
"Malay: Kuda itu kelihatan seksi"

"English: your father is a sexy policeman. "
"Malay: ayah kamu merupakan seorang polis yang seksi."

"English: I like cows. "
"Malay: Saya suka lembu."

"English: This company provide great opportunity. "
"Malay: syarikat ini memberikan peluang yang besar."

"English: I love coral reefs"
"Malay: saya suka terumbu karang"

"English: Albert Einstein is a theoretical physicist"
"Malay: Albert Einstein ialah seorang ahli fizik teori"

)

User_input = "Saya amat suka tempat ini"
Lang_en = "Malay:"
True_input = Lang_en + User_input

input0 = [[Training_phrases + True_input ],]
inputs = prepare_inputs(input0)
result = client.infer(MODEl_GPTJ_FASTERTRANSFORMER, inputs)
output0 = result.as_numpy("OUTPUT_0")

output0 = np.array2string(output0)


k = output0.find(User_input)


k2 = output0.find('English', k)




m1 = output0.find( 'Malay',(k+ len(User_input ) ))
offset = k + len('English') + len(User_input)
m2 = output0.find( 'English',(offset )) #these three line is here because the AI will spit nonesense
#after it is done translating, so we have to pick it up when the AI 

m = min(m1,m2)


print(output0[k2:m])


In [ ]:
def Translate( User_input, input_lang):
    
    Training_phrases = (
    "Malay: Di mana stesen kereta api terdekat?"
    "English: Where is the nearest train station? "
    
    "Malay: Berapakah harga tiket kereta api ke warsaw? "
    "English: How much is a train ticket to Warsaw?"
    
    "Malay: Bolehkah anda cakap bahasa melayu?"
    "English: can you speak malay?"
    
    "Malay: Jepun merupakan sesuatu bandar di dalam Tokyo"
    "English: Japan is a city in Tokyo"
    
    "Malay: Hang Tuah ialah seorang pahlawan legenda berbangsa Melayu pada masa pemerintahan Kesultanan Melayu Melaka di abad ke-15 "
    "English: Hang Tuah was a legendary warrior of the Malay race during the reign of the Malacca Sultanate in the 15th century "

    "English: The horse looks sexy "
    "Malay: Kuda itu kelihatan seksi"
    
    "English: your father is a sexy policeman. "
    "Malay: ayah kamu merupakan seorang polis yang seksi."
        
    "English: I like cows. "
    "Malay: Saya suka lembu."
        
    "English: This company provide great opportunity. "
    "Malay: syarikat ini memberikan peluang yang besar."
    
    "English: I love coral reefs"
    "Malay: saya suka terumbu karang"
        
    "English: Albert Einstein is a theoretical physicist"
    "Malay: Albert Einstein ialah seorang ahli fizik teori"
    
)
    if input_lang == 'EN' :
        Lang_en = "English:"
        True_input = Lang_en + User_input

        input0 = [[Training_phrases + True_input ],]
        inputs = prepare_inputs(input0)
        result = client.infer(MODEl_GPTJ_FASTERTRANSFORMER, inputs)
        output0 = result.as_numpy("OUTPUT_0")

        output0 = np.array2string(output0)

        k = output0.find(User_input)

        k2 = output0.find('Malay', k)

        m1 = output0.find( 'English',(k+ len(User_input ) ))
        offset = k + len('Malay') + len(User_input)
        m2 = output0.find( 'Malay',(offset )) #these three line is here because the AI will spit nonesense
        #after it is done translating, so we have to pick it up when the AI 

        m = min(m1,m2)


        print(output0[k2:m])


            

    if input_lang == 'BM' :
        try:
            Lang_bm = "Malay:"
            True_input = Lang_bm + User_input
            
            input0 = [[Training_phrases + True_input ],]
            inputs = prepare_inputs(input0)
            result = client.infer(MODEl_GPTJ_FASTERTRANSFORMER, inputs)
            output0 = result.as_numpy("OUTPUT_0")
            
            output0 = np.array2string(output0)
            k = output0.find(User_input)
            
            k2 = output0.find('English', k)
            
            m1 = output0.find( 'Malay',(k+ len(User_input ) ))
            offset = k + len('English') + len(User_input)
            m2 = output0.find( 'English',(offset ))
            
            m = min(m1,m2)
            print(k2,m1,m2,m)
            print(output0[k2:m])
            
        except:
            print("User_input have to be a string")
        
        
    

In [ ]:
Translate("saya amat suka tempat ini","BM")